#  Spike PAtterns DEtection using SPADE


In [1]:
import numpy as np
import quantities as pq
import neo
import elephant
import viziphant
np.random.seed(4542)

## introducing heterogeneous delays

the function developed in the scan + a shift in time

In [2]:

A_sync = 0.02
shift = 0.01
A = [0]+[1.-A_sync]+[0]*8+[A_sync]
spiketrains = elephant.spike_train_generation.compound_poisson_process(
                             rate=5*pq.Hz, A=A, shift=shift*pq.s, t_stop=10*pq.s)
                             


Add a shift:

In [3]:
spiketrains[0].times + 4.2*pq.s

array([ 4.54268786,  4.56949328,  5.42288289,  5.48848231,  5.83890298,
        7.082554  ,  7.23500582,  7.23802362,  7.61871493,  7.6674776 ,
        8.03221782,  8.05683958,  8.37877339,  8.38897297,  8.70248868,
        9.31693589,  9.45178525,  9.52463345, 10.5950124 , 10.70183048,
       11.12921023, 11.18969509, 11.30194166, 11.35880212, 11.76574189,
       11.98102997, 12.44154892, 12.51100792, 12.69858892, 12.9055502 ,
       13.11660239, 13.17491733, 13.23408406, 13.23837955, 13.3545316 ,
       13.59283235, 13.72926468, 13.81556317, 14.11329785]) * s

Wrap around time:

In [4]:
np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s))

array([0.5950124 , 0.70183048, 1.12921023, 1.18969509, 1.30194166,
       1.35880212, 1.76574189, 1.98102997, 2.44154892, 2.51100792,
       2.69858892, 2.9055502 , 3.11660239, 3.17491733, 3.23408406,
       3.23837955, 3.3545316 , 3.59283235, 3.72926468, 3.81556317,
       4.11329785, 4.54268786, 4.56949328, 5.42288289, 5.48848231,
       5.83890298, 7.082554  , 7.23500582, 7.23802362, 7.61871493,
       7.6674776 , 8.03221782, 8.05683958, 8.37877339, 8.38897297,
       8.70248868, 9.31693589, 9.45178525, 9.52463345]) * s

Sort in time:

In [5]:
np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s))

array([0.5950124 , 0.70183048, 1.12921023, 1.18969509, 1.30194166,
       1.35880212, 1.76574189, 1.98102997, 2.44154892, 2.51100792,
       2.69858892, 2.9055502 , 3.11660239, 3.17491733, 3.23408406,
       3.23837955, 3.3545316 , 3.59283235, 3.72926468, 3.81556317,
       4.11329785, 4.54268786, 4.56949328, 5.42288289, 5.48848231,
       5.83890298, 7.082554  , 7.23500582, 7.23802362, 7.61871493,
       7.6674776 , 8.03221782, 8.05683958, 8.37877339, 8.38897297,
       8.70248868, 9.31693589, 9.45178525, 9.52463345]) * s

https://neo.readthedocs.io/en/stable/core.html?highlight=spiketrain#example-spiketrain

In [6]:
neo.SpikeTrain(np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s)), t_start=0*pq.s, t_stop=10*pq.s)



SpikeTrain

In [7]:
for i_st in range(10):
    delay = (i_st*slope)*pq.s
    spike_times = np.sort(np.mod(spiketrains[i_st].times + delay, t_stop))
    spiketrains[i_st] = neo.SpikeTrain(spike_times, t_start=0*pq.s, t_stop=t_stop)

NameError: name 'slope' is not defined

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 7))
plt.eventplot([spiketrains[i].magnitude for i in range(len(spiketrains))], linelengths=0.75, color='black')
plt.xlabel('Time (ms)', fontsize=16)
plt.ylabel('Neuron')
plt.title("Raster plot");

All in one function:

All in one function:

In [ ]:
t_stop = 10*pq.s
def generate_and_test(A_sync=A_sync, shift=shift, winlen=1, slope=1., t_stop=t_stop):

    A = [0]+[1.-A_sync]+[0]*8+[A_sync]
    spiketrains = elephant.spike_train_generation.compound_poisson_process(
                             rate=5*pq.Hz, A=A, shift=shift*pq.s, t_stop=t_stop)
    for i_st in range(10):
        delay = (i_st*slope)*pq.s
        spike_times = np.sort(np.mod(spiketrains[i_st].times + delay, t_stop))
        spiketrains[i_st] = neo.SpikeTrain(spike_times, t_start=0*pq.s, t_stop=t_stop)

    for i in range(90):
        spiketrains.append(elephant.spike_train_generation.homogeneous_poisson_process(
            rate=5*pq.Hz, t_stop=10*pq.s))
    patterns = elephant.spade.spade(
                                spiketrains=spiketrains, binsize=1*pq.ms, winlen=winlen, min_spikes=3,
                                n_surr=100, dither=5*pq.ms,
                                psr_param=[0,0,0],
                                output_format='patterns')['patterns']
    return patterns


* changing the synchrony probability to check when SPADE will begin to fail:


In [ ]:
for slope_ in np.logspace(-1, 1, 10, base=10):
    patterns = generate_and_test(slope=slope_)
    print(slope_, patterns)

In [ ]:
for slope_ in np.logspace(-1, 1, 10, base=10):
    patterns = generate_and_test(slope=slope_, winlen=10)
    print(slope_, patterns)